In [1]:
import os
import pandas as pd
import datetime
import re
import logging

# Setup basic logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def list_chat_files(date_directory):
    chat_files = []
    for date_folder in os.listdir(date_directory):
        date_path = os.path.join(date_directory, date_folder)
        if os.path.isdir(date_path):
            for team_folder in os.listdir(date_path):
                if team_folder != "KAM":
                    continue
                
                team_path = os.path.join(date_path, team_folder)
                if os.path.isdir(team_path):
                    for person_folder in os.listdir(team_path):
                        person_path = os.path.join(team_path, person_folder)
                        if os.path.isdir(person_path):
                            for file in os.listdir(person_path):
                                if file.endswith('.txt'):
                                    chat_files.append(os.path.join(person_path, file))
    logging.debug(f"Chat files listed: {chat_files}")
    return chat_files

def parse_chat_file(file_path, expected_date_minus_one):
    chat_data = []
    last_non_person_time = None  # Tracks the time of the last non-person message
    delay_count = 0  # To count the number of delays

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            message_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*?): (.*)', line)
            system_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*)', line)
            if message_match:
                date_time_str, sender, message = message_match.groups()
            elif system_match:
                date_time_str, info = system_match.groups()
                sender = None
            else:
                continue

            date_time = pd.to_datetime(date_time_str, format='%d/%m/%y, %I:%M %p')

            if date_time.date() != expected_date_minus_one:
                continue

            is_person = sender is not None and re.match(r'^[+\d\s-]+$', sender) is None

            # Calculate delay
            delay = False
            if is_person and last_non_person_time:
                diff = date_time - last_non_person_time
                delay = diff.total_seconds() > 900  # 15 minutes in seconds
                if delay:
                    delay_count += 1

            chat_data.append((date_time, sender, is_person, delay))

            # Update last_non_person_time for non-person messages
            if not is_person:
                last_non_person_time = date_time

    logging.debug(f"File parsed: {file_path}. Delays detected: {delay_count}")
    return chat_data


def create_template_dataframe():
    times = [datetime.datetime(2000, 1, 1, 0, 0) + datetime.timedelta(minutes=1 * i) for i in range(1440)]
    intervals = [time.strftime('%I:%M %p') for time in times]
    df = pd.DataFrame(index=pd.to_datetime(intervals).strftime('%I:%M %p').unique())  # Ensure unique intervals
    return df

def populate_dataframe(df, parsed_data, group_name):
    # Define new column names
    person_col = f"{group_name}_person"
    others_col = f"{group_name}_others"
    delay_col = f"{group_name}_delay"

    # Initialize new columns
    if person_col not in df.columns:
        df[person_col] = 0
    if others_col not in df.columns:
        df[others_col] = 0
    if delay_col not in df.columns:
        df[delay_col] = 0

    # Populate the new columns with parsed data
    for date_time, sender, is_person, delay in parsed_data:
        interval_index = min((date_time.hour * 60 + date_time.minute) // 1, 1439)
        interval = df.index[interval_index]

        if is_person:
            df.at[interval, person_col] = 1
        else:
            df.at[interval, others_col] = 1

        if delay:
            df.at[interval, delay_col] = 1

        logging.debug(f"Updated DataFrame at {interval} for {group_name}: Person={is_person}, Delay={delay}")

    # Update active_chat column
    if 'active_chat' not in df.columns:
        df['active_chat'] = 0

    relevant_columns = [person_col, others_col]
    df['active_chat'] = df[relevant_columns].any(axis=1).astype(int)

    return df

def extract_group_name(file_path):
    group_name = os.path.basename(file_path).replace('WhatsApp Chat with ', '').split('.')[0]
    group_name = re.sub(r'\(\d+\)$', '', group_name)  # Remove any numbers in parentheses at the end
    return group_name  # Removed the extra "_person" suffix

date_directory = "C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat Folder from Drive\\drive-download-20231205T080950Z-001"
chat_files = list_chat_files(date_directory)
dataframes = {}

for file in chat_files:
    parts = file.split(os.sep)
    date_folder, person = parts[-4], parts[-2]

    try:
        folder_date = pd.to_datetime(date_folder, format='%Y-%m-%d').date()
    except ValueError:
        continue

    expected_date_minus_one = folder_date - datetime.timedelta(days=1)
    key = f"{folder_date.strftime('%Y-%m-%d')}_{person}"

    # Extract group_name using the dedicated function
    group_name = extract_group_name(file)

    if key not in dataframes:
        dataframes[key] = create_template_dataframe()
    
    # Get parsed_data without expecting group_name in return
    parsed_data = parse_chat_file(file, expected_date_minus_one)
    
    dataframes[key] = populate_dataframe(dataframes[key], parsed_data, group_name)
    logging.debug(f"Dataframe created for key: {key}")

# Example to show a dataframe
example_key = next(iter(dataframes))  # Just for demonstration
logging.debug(f"Example dataframe for key {example_key}: \n{dataframes[example_key]}")


csv_save_directory = "C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat CSVs"
os.makedirs(csv_save_directory, exist_ok=True)

# Saving each DataFrame as a CSV
for key, df in dataframes.items():
    csv_file_path = os.path.join(csv_save_directory, f"{key}.csv")
    df.to_csv(csv_file_path)
    print(f"file saved as {key}")


2023-12-05 14:19:33,484 - DEBUG - Chat files listed: ['C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat Folder from Drive\\drive-download-20231205T080950Z-001\\2023-12-05\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Aaron 2K23OCT1666R.txt', 'C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat Folder from Drive\\drive-download-20231205T080950Z-001\\2023-12-05\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Adella 2K22FEB2869.txt', 'C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat Folder from Drive\\drive-download-20231205T080950Z-001\\2023-12-05\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Agape 2K23NOV2370R.txt', 'C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat Folder from Drive\\drive-download-20231205T080950Z-001\\2023-12-05\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Audrey 2K23OCT0992.txt', 'C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat Folder from Drive\\drive-download-20231205T080950Z-001\\2023-12-05\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Brooke 2K23MAY0260.txt', 'C:\\Users\\maurice\\

2023-12-05 14:19:33,613 - DEBUG - File parsed: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231205T080950Z-001\2023-12-05\KAM\Ashi_Edoofa\WhatsApp Chat with Aaron 2K23OCT1666R.txt. Delays detected: 1
2023-12-05 14:19:33,617 - DEBUG - Updated DataFrame at 10:58 AM for Aaron 2K23OCT1666R: Person=False, Delay=False
2023-12-05 14:19:33,618 - DEBUG - Updated DataFrame at 06:10 PM for Aaron 2K23OCT1666R: Person=True, Delay=True
2023-12-05 14:19:33,620 - DEBUG - Dataframe created for key: 2023-12-05_Ashi_Edoofa
2023-12-05 14:19:33,637 - DEBUG - File parsed: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231205T080950Z-001\2023-12-05\KAM\Ashi_Edoofa\WhatsApp Chat with Adella 2K22FEB2869.txt. Delays detected: 0
2023-12-05 14:19:33,637 - DEBUG - Updated DataFrame at 06:15 AM for Adella 2K22FEB2869: Person=False, Delay=False
2023-12-05 14:19:33,668 - DEBUG - Dataframe created for key: 2023-12-05_Ashi_Edoofa
2023-12-05 14:

2023-12-05 14:19:33,871 - DEBUG - Updated DataFrame at 07:06 PM for Darlington 2K22JAN0402: Person=False, Delay=False
2023-12-05 14:19:33,871 - DEBUG - Updated DataFrame at 07:07 PM for Darlington 2K22JAN0402: Person=True, Delay=False
2023-12-05 14:19:33,871 - DEBUG - Updated DataFrame at 07:08 PM for Darlington 2K22JAN0402: Person=False, Delay=False
2023-12-05 14:19:33,871 - DEBUG - Dataframe created for key: 2023-12-05_Ashi_Edoofa
2023-12-05 14:19:33,909 - DEBUG - File parsed: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231205T080950Z-001\2023-12-05\KAM\Ashi_Edoofa\WhatsApp Chat with David 2K23SEP2834.txt. Delays detected: 1
2023-12-05 14:19:33,911 - DEBUG - Updated DataFrame at 09:18 AM for David 2K23SEP2834: Person=False, Delay=False
2023-12-05 14:19:33,912 - DEBUG - Updated DataFrame at 12:51 PM for David 2K23SEP2834: Person=False, Delay=False
2023-12-05 14:19:33,912 - DEBUG - Updated DataFrame at 12:58 PM for David 2K23SEP2834: Person=True,

2023-12-05 14:19:34,087 - DEBUG - Updated DataFrame at 06:50 PM for Kelvin 2K23NOV1626R: Person=False, Delay=False
2023-12-05 14:19:34,087 - DEBUG - Updated DataFrame at 07:23 PM for Kelvin 2K23NOV1626R: Person=True, Delay=True
2023-12-05 14:19:34,087 - DEBUG - Updated DataFrame at 07:34 PM for Kelvin 2K23NOV1626R: Person=False, Delay=False
2023-12-05 14:19:34,101 - DEBUG - Updated DataFrame at 07:50 PM for Kelvin 2K23NOV1626R: Person=True, Delay=True
2023-12-05 14:19:34,101 - DEBUG - Updated DataFrame at 07:50 PM for Kelvin 2K23NOV1626R: Person=True, Delay=True
2023-12-05 14:19:34,101 - DEBUG - Updated DataFrame at 07:50 PM for Kelvin 2K23NOV1626R: Person=True, Delay=True
2023-12-05 14:19:34,102 - DEBUG - Updated DataFrame at 07:51 PM for Kelvin 2K23NOV1626R: Person=True, Delay=True
2023-12-05 14:19:34,102 - DEBUG - Updated DataFrame at 08:04 PM for Kelvin 2K23NOV1626R: Person=False, Delay=False
2023-12-05 14:19:34,103 - DEBUG - Updated DataFrame at 08:04 PM for Kelvin 2K23NOV1626R: P

2023-12-05 14:19:34,185 - DEBUG - Updated DataFrame at 01:16 PM for Kudakwashe 2K23NOV2061R: Person=True, Delay=False
2023-12-05 14:19:34,185 - DEBUG - Updated DataFrame at 01:16 PM for Kudakwashe 2K23NOV2061R: Person=True, Delay=False
2023-12-05 14:19:34,185 - DEBUG - Updated DataFrame at 01:20 PM for Kudakwashe 2K23NOV2061R: Person=False, Delay=False
2023-12-05 14:19:34,186 - DEBUG - Updated DataFrame at 01:22 PM for Kudakwashe 2K23NOV2061R: Person=True, Delay=False
2023-12-05 14:19:34,186 - DEBUG - Updated DataFrame at 01:22 PM for Kudakwashe 2K23NOV2061R: Person=True, Delay=False
2023-12-05 14:19:34,186 - DEBUG - Updated DataFrame at 01:22 PM for Kudakwashe 2K23NOV2061R: Person=True, Delay=False
2023-12-05 14:19:34,187 - DEBUG - Updated DataFrame at 01:23 PM for Kudakwashe 2K23NOV2061R: Person=False, Delay=False
2023-12-05 14:19:34,187 - DEBUG - Updated DataFrame at 01:24 PM for Kudakwashe 2K23NOV2061R: Person=False, Delay=False
2023-12-05 14:19:34,189 - DEBUG - Updated DataFrame a

...

2023-12-05 14:19:42,153 - DEBUG - Updated DataFrame at 01:37 PM for Tafadzwa 2K22DEC2028: Person=True, Delay=False
2023-12-05 14:19:42,153 - DEBUG - Updated DataFrame at 01:45 PM for Tafadzwa 2K22DEC2028: Person=False, Delay=False
2023-12-05 14:19:42,153 - DEBUG - Updated DataFrame at 01:57 PM for Tafadzwa 2K22DEC2028: Person=True, Delay=False
2023-12-05 14:19:42,168 - DEBUG - Updated DataFrame at 05:11 PM for Tafadzwa 2K22DEC2028: Person=False, Delay=False
2023-12-05 14:19:42,168 - DEBUG - Updated DataFrame at 05:29 PM for Tafadzwa 2K22DEC2028: Person=True, Delay=True
2023-12-05 14:19:42,169 - DEBUG - Updated DataFrame at 05:48 PM for Tafadzwa 2K22DEC2028: Person=False, Delay=False
2023-12-05 14:19:42,169 - DEBUG - Updated DataFrame at 05:56 PM for Tafadzwa 2K22DEC2028: Person=True, Delay=False
2023-12-05 14:19:42,174 - DEBUG - Dataframe created for key: 2023-12-05_Shivjeet Edoofa
2023-12-05 14:19:42,217 - DEBUG - File parsed: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder fr

2023-12-05 14:19:42,402 - DEBUG - Updated DataFrame at 01:43 PM for Talent 2K23AUG2354: Person=True, Delay=False
2023-12-05 14:19:42,418 - DEBUG - Dataframe created for key: 2023-12-05_Shivjeet Edoofa
2023-12-05 14:19:42,437 - DEBUG - File parsed: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231205T080950Z-001\2023-12-05\KAM\Shivjeet Edoofa\WhatsApp Chat with TANAKA 2K22DEC1748.txt. Delays detected: 1
2023-12-05 14:19:42,437 - DEBUG - Updated DataFrame at 05:58 PM for TANAKA 2K22DEC1748: Person=False, Delay=False
2023-12-05 14:19:42,437 - DEBUG - Updated DataFrame at 06:44 PM for TANAKA 2K22DEC1748: Person=True, Delay=True
2023-12-05 14:19:42,437 - DEBUG - Updated DataFrame at 06:45 PM for TANAKA 2K22DEC1748: Person=False, Delay=False
2023-12-05 14:19:42,437 - DEBUG - Updated DataFrame at 06:55 PM for TANAKA 2K22DEC1748: Person=True, Delay=False
2023-12-05 14:19:42,437 - DEBUG - Dataframe created for key: 2023-12-05_Shivjeet Edoofa
2023-12-05 14:1

2023-12-05 14:19:42,703 - DEBUG - Updated DataFrame at 07:03 PM for Tawananyasha 2K23NOV2371R: Person=True, Delay=False
2023-12-05 14:19:42,703 - DEBUG - Updated DataFrame at 07:03 PM for Tawananyasha 2K23NOV2371R: Person=True, Delay=False
2023-12-05 14:19:42,703 - DEBUG - Updated DataFrame at 07:03 PM for Tawananyasha 2K23NOV2371R: Person=True, Delay=False
2023-12-05 14:19:42,704 - DEBUG - Updated DataFrame at 07:03 PM for Tawananyasha 2K23NOV2371R: Person=True, Delay=False
2023-12-05 14:19:42,704 - DEBUG - Updated DataFrame at 07:03 PM for Tawananyasha 2K23NOV2371R: Person=True, Delay=False
2023-12-05 14:19:42,705 - DEBUG - Updated DataFrame at 07:03 PM for Tawananyasha 2K23NOV2371R: Person=True, Delay=False
2023-12-05 14:19:42,705 - DEBUG - Updated DataFrame at 07:08 PM for Tawananyasha 2K23NOV2371R: Person=False, Delay=False
2023-12-05 14:19:42,706 - DEBUG - Updated DataFrame at 07:19 PM for Tawananyasha 2K23NOV2371R: Person=False, Delay=False
2023-12-05 14:19:42,707 - DEBUG - Upda

2023-12-05 14:19:42,964 - DEBUG - Updated DataFrame at 07:29 PM for _Dalitso 2K23NOV0232: Person=True, Delay=True
2023-12-05 14:19:42,964 - DEBUG - Updated DataFrame at 07:59 PM for _Dalitso 2K23NOV0232: Person=False, Delay=False
2023-12-05 14:19:42,965 - DEBUG - Updated DataFrame at 08:06 PM for _Dalitso 2K23NOV0232: Person=True, Delay=False
2023-12-05 14:19:42,965 - DEBUG - Updated DataFrame at 08:17 PM for _Dalitso 2K23NOV0232: Person=False, Delay=False
2023-12-05 14:19:42,965 - DEBUG - Updated DataFrame at 08:58 PM for _Dalitso 2K23NOV0232: Person=True, Delay=True
2023-12-05 14:19:42,966 - DEBUG - Updated DataFrame at 09:19 PM for _Dalitso 2K23NOV0232: Person=False, Delay=False
2023-12-05 14:19:42,966 - DEBUG - Updated DataFrame at 09:20 PM for _Dalitso 2K23NOV0232: Person=True, Delay=False
2023-12-05 14:19:42,967 - DEBUG - Updated DataFrame at 09:21 PM for _Dalitso 2K23NOV0232: Person=False, Delay=False
2023-12-05 14:19:42,969 - DEBUG - Dataframe created for key: 2023-12-05_Shivje

file saved as 2023-12-05_Ashi_Edoofa
file saved as 2023-12-05_Kirti Edoofa
file saved as 2023-12-05_Milan_Edoofa
file saved as 2023-12-05_Shivjeet Edoofa


In [4]:
import os
import pandas as pd
import datetime
import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Function to list all chat files in the directory structure
def list_chat_files(date_directory):
    chat_files = []
    for date_folder in os.listdir(date_directory):
        date_path = os.path.join(date_directory, date_folder)
        if os.path.isdir(date_path):
            for team_folder in os.listdir(date_path):
                team_path = os.path.join(date_path, team_folder)
                if os.path.isdir(team_path):
                    for person_folder in os.listdir(team_path):
                        person_path = os.path.join(team_path, person_folder)
                        if os.path.isdir(person_path):
                            for file in os.listdir(person_path):
                                if file.endswith('.txt'):
                                    chat_files.append(os.path.join(person_path, file))
    return chat_files

def parse_chat_file(file_path, expected_date_minus_one):
    chat_data = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            message_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*?): (.*)', line)
            if message_match:
                date_time_str, sender, _ = message_match.groups()
                date_time = pd.to_datetime(date_time_str, format='%d/%m/%y, %I:%M %p')

                if date_time.date() != expected_date_minus_one:
                    continue

                is_person = sender is not None and re.match(r'^[+\d\s-]+$', sender) is None
                chat_data.append((date_time, sender, is_person))

    return chat_data

# Function to create a template dataframe
def create_template_dataframe():
    times = [datetime.datetime(2000, 1, 1, 0, 0) + datetime.timedelta(minutes=1 * i) for i in range(1440)]
    intervals = [time.strftime('%I:%M %p') for time in times]
    df = pd.DataFrame(index=intervals)
    return df

def populate_dataframe(df, parsed_data, start_column_index):
    new_columns = {}  # Dictionary to hold new data before concatenation

    for entry in parsed_data:
        date_time, sender, is_person = entry
        interval_index = min((date_time.hour * 60 + date_time.minute) // 1, 1439)
        interval = df.index[interval_index]

        # Initialize columns in new_columns dictionary if not exist
        if start_column_index not in new_columns:
            new_columns[start_column_index] = pd.Series(0, index=df.index)
        if start_column_index + 1 not in new_columns:
            new_columns[start_column_index + 1] = pd.Series(0, index=df.index)

        # Populate the new_columns dictionary
        if is_person:
            new_columns[start_column_index].at[interval] = 1
        else:
            new_columns[start_column_index + 1].at[interval] = 1

    # Concatenate new columns to the DataFrame at once
    df = pd.concat([df, pd.DataFrame(new_columns)], axis=1)

    return df, start_column_index + 2


def process_person_chats(chat_files):
    dataframes = {}
    for file in chat_files:
        parts = file.split(os.sep)
        date_folder, person = parts[-4], parts[-2]

        try:
            folder_date = pd.to_datetime(date_folder, format='%Y-%m-%d').date()
        except ValueError:
            continue

        expected_date_minus_one = folder_date - datetime.timedelta(days=1)
        key = f"{folder_date.strftime('%Y-%m-%d')}_{person}"

        if key not in dataframes:
            dataframes[key] = create_template_dataframe()
            start_column_index = 0
        else:
            if not dataframes[key].columns.empty:
                start_column_index = max(dataframes[key].columns) + 1
            else:
                start_column_index = 0

        parsed_data = parse_chat_file(file, expected_date_minus_one)
        dataframes[key], start_column_index = populate_dataframe(dataframes[key], parsed_data, start_column_index)

    return dataframes


print("analysis complete. Generating Graphs...")


# Function to create bar and trend graphs for each person
def create_graphs(df, person_identifier, base_directory):
    graph_directory = os.path.join(base_directory, "Graphs")
    os.makedirs(graph_directory, exist_ok=True)

    # Sum the values across all even columns for each minute
    person_chat_activity = df.iloc[:, 0::2].sum(axis=1)
    trend_data = df.iloc[:, 1::2].sum(axis=1)

    # Find the first and last non-zero indices for chats
    non_zero_indices = person_chat_activity[person_chat_activity > 0].index
    first_chat_time = non_zero_indices[0] if not non_zero_indices.empty else df.index[0]
    last_chat_time = non_zero_indices[-1] if not non_zero_indices.empty else df.index[-1]

    print(f"Generating graph for {person_identifier}")  # Debug line
    fig, ax = plt.subplots(figsize=(30, 10))  # Increased figure size for more stretch
    fig.patch.set_facecolor('black')
    ax.set_facecolor('black')
    ax.xaxis.label.set_color('white')
    ax.yaxis.label.set_color('white')
    ax.title.set_color('white')

    # Set the colors of the tick labels
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')

    # Plot the bar and scatter with the specified colors and labels
    ax.bar(df.index, person_chat_activity, color='red', width=1, label=f'Person {person_identifier}')
    ax.plot(df.index, trend_data, color='white', linestyle=':', label='Student')
    #ax.scatter(df.index[trend_data > 0], trend_data[trend_data > 0], color='white', s=8, label='Student')
    
    # Draw white lines for the axes
    ax.axhline(0, color='white', linewidth=3)
    ax.axvline(first_chat_time, color='white', linewidth=3)
    
    # Rotate x-axis labels to prevent overlap and increase label font sizes
    plt.xticks(rotation=90, fontsize=12)
    
    # Set y-axis to show every integer tick
    plt.yticks(np.arange(0, 11, 1), fontsize=12)

    # Set x-axis to show the range from the first chat to the last chat
    ax.set_xlim(first_chat_time, last_chat_time)
    ax.xaxis.set_major_locator(ticker.MaxNLocator(96))  # Set locator for 15-minute intervals

    # Set y-axis dynamic range based on the maximum chat activity with a buffer
    ax.set_ylim(0, 11)

    # Increasing font size for labels and title
    ax.set_xlabel('Time', fontsize=12)
    ax.set_ylabel('Number of Chats', fontsize=12)
    ax.set_title(f'Chat Activity for {person_identifier}', fontsize=14)

    # Create and set the legend
    legend = ax.legend(facecolor='white', edgecolor='red', fontsize=12, fancybox=True)
    for text in legend.get_texts():
        text.set_color('black')
        text.set_weight('bold')

    # Saving the graph
    graph_file_name = f"{person_identifier}.png"
    plt.savefig(os.path.join(graph_directory, graph_file_name), format='png', dpi=300, bbox_inches='tight')  # Save with tight bounding box
    print(f"Graph saved as {graph_file_name}")  # Debug line

    plt.close(fig)


# Main script
date_directory = "C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat Folder from Drive\\drive-download-20231205T080950Z-001"
chat_files = list_chat_files(date_directory)
person_dataframes = process_person_chats(chat_files)

# Generating graphs for each DataFrame
for person_identifier, df in person_dataframes.items():
    create_graphs(df, person_identifier, date_directory)

analysis complete. Generating Graphs...


2023-12-05 14:37:57,412 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-05 14:37:57,495 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.


Generating graph for 2023-12-05_Aditi_Edoofa


2023-12-05 14:37:58,523 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-05 14:37:58,613 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-05 14:37:58,624 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-05 14:37:58,624 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-05 14:38:02,055 - INFO - Using categorical units to plot a list of strings that are all parsable as 

Graph saved as 2023-12-05_Aditi_Edoofa.png
Generating graph for 2023-12-05_Ananya_Edoofa


...

Graph saved as 2023-12-05_Shashwat Edoofa 2.png
Generating graph for 2023-12-05_Shubham Madhwal


2023-12-05 14:40:04,795 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-05 14:40:06,066 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-05 14:40:06,167 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-05 14:40:06,179 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-05 14:40:06,179 - INFO - Using categorical units to plot a list of strings that are all parsable as 

Graph saved as 2023-12-05_Shubham Madhwal.png
Generating graph for 2023-12-05_Tushti


2023-12-05 14:40:13,185 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-05 14:40:13,252 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-05 14:40:13,268 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-05 14:40:13,269 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2023-12-05 14:40:14,392 - INFO - Using categorical units to plot a list of strings that are all parsable as 

Graph saved as 2023-12-05_Tushti.png


In [5]:
def save_dataframes_to_csv(dataframes, base_directory):
    save_directory = os.path.join(base_directory, "CSVs")
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    for person_identifier, df in dataframes.items():
        file_name = f"{person_identifier}.csv"
        file_path = os.path.join(save_directory, file_name)
        df.to_csv(file_path)
        print(f"Saved DataFrame for {person_identifier} to {file_path}")

# Usage
save_dataframes_to_csv(person_dataframes, date_directory)


Saved DataFrame for 2023-12-05_Aditi_Edoofa to C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231205T080950Z-001\CSVs\2023-12-05_Aditi_Edoofa.csv
Saved DataFrame for 2023-12-05_Ananya_Edoofa to C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231205T080950Z-001\CSVs\2023-12-05_Ananya_Edoofa.csv
Saved DataFrame for 2023-12-05_Jasmine_Edoofa to C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231205T080950Z-001\CSVs\2023-12-05_Jasmine_Edoofa.csv
Saved DataFrame for 2023-12-05_Saloni_Edoofa to C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231205T080950Z-001\CSVs\2023-12-05_Saloni_Edoofa.csv
Saved DataFrame for 2023-12-05_Sharda_Edoofa to C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231205T080950Z-001\CSVs\2023-12-05_Sharda_Edoofa.csv
Saved DataFrame for 2023-12-05_Ashi_Edoofa to C:\Users\maurice\Documents\Ch